In [1]:
from bnlp import NLTKTokenizer
import pandas as pd
import pickle
import numpy as np
from gensim.models import Phrases
from gensim.models.phrases import Phraser
import warnings
warnings.filterwarnings('ignore')

c:\users\administrator\appdata\local\programs\python\python38\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
df = pd.read_csv("bangla_news.csv")
pd.set_option("display.max_colwidth",-1)

fobj = open("recom.pkl","rb")
lda = pickle.load(fobj)
fobj.close()

fileob = open("dtm.pkl","rb")
token = pickle.load(fileob)
fileob.close()

filed = open("dict.pkl","rb")
dictionary = pickle.load(filed)
filed.close()

filebi = open("bigram.pkl","rb")
bigram = pickle.load(filebi)
filebi.close()

In [3]:
def todict(topics):
  dict = {}
  for j in range(len(topics)):
    dict[topics[j][0]] = topics[j][1]
  return dict

In [4]:
def similarity(li1,li2):
    return np.dot(li1,li2)/(np.linalg.norm(li1)*np.linalg.norm(li2))

In [5]:
def make_bigram(text):
    return [bigram[word] for word in text]

In [6]:
def recommand_news():
    text = input("ENTER ARTICLE :- ")
    
    tokenized = NLTKTokenizer().word_tokenize(text)
    
    tokens = token.copy()
    tokens.append(tokenized)
    
    #APPLYING BIGRAM MODEL
    tokenize_text = make_bigram(tokens)
    
    # FEEDING DATA INTO TRAINED LDA MODEL
    dtms = [dictionary.doc2bow(word) for word in tokenize_text]
    topics = list(lda[dtms])
    
    length = len(topics)-1
    dic = todict(topics[length])
    
    # DISCARDING TOPICS WHICH DOES NOT HAVE 5% CONTRIBUTION
    predict_topic = dict({})
    for key in dic.keys():
        if dic[key] >= 0.05:
            predict_topic[key] = dic[key]  
            
    topic_li = [todict(topic) for topic in topics[:-1]]
    
    # FINDING DOCUMENTS WHICH HAVE TOPICS MOST SIMILER TO THE GIVEN DOCUMENT
    c = 0
    divid = 1
    while c == 0:
        rl = []
        rt = []
        for i in range(len(topic_li)):
            l = len(predict_topic.keys())
            d = dict({})
            for j in topic_li[i].keys():
                if j in predict_topic.keys() and topic_li[i][j] >= predict_topic[j]/divid:
                    l-=1
                    d[j] = topic_li[i][j]
            if l == 0:
                rl.append(i)
                rt.append(similarity(list(predict_topic.values()),list(d.values())))
        
        if len(rt) < 5:
            c = 0
            divid+=0.1
        else:
            c = 1
        
    rlist = list(np.argsort(rt))
    rlist.reverse()

    print(divid)
    return [str(predict_topic)+"\n"+str(topic_li[rl[i]])+"\n"+str(rt[i])+"\n"+df.iloc[rl[i]].article for i in rlist[:10]]

In [7]:
while True:

  rec = recommand_news()

  i=1
  for article in rec:
    print(f"ARTICLE - {i} => ")
    print("=====================================================")
    print(article)
    print("=====================================================")
    i+=1
    if i > 10:
      break

  while True:
    print("\n\n-------------------------------------------------")
    ch = input("DO YOU WANT TO CHECK FOR ANOTHER ARTICLE (Y/N) :- ")
    
    if ch == 'N' or ch == 'Y' or ch == 'n' or ch == 'y':
      break
    else:
      print("PLEASE CHOOSE CORRECTLY......")
    print("-------------------------------------------------\n\n")
  if ch == 'N' or ch == 'n':
    break

ENTER ARTICLE :- "বেঙ্গালুরুতে দিন-রাতের টেস্টে প্রথম ইনিংসে ৫ ও দ্বিতীয় ইনিংসে ৩ উইকেট নিয়েছেন ভারতীয় পেসার যশপ্রীত বুমরা। এই ৮ উইকেটের সৌজন্যে আইসিসি টেস্ট বোলারদের তালিকায় উন্নতি হয়েছে তাঁর। ১০ নম্বর থেকে এক লাফে ৪ নম্বরে উঠে এসেছেন বুমরা। তাঁর পয়েন্ট ৮৩০।  টেস্ট বোলারদের তালিকায় শীর্ষে রয়েছেন অস্ট্রেলিয়ার অধিনায়ক প্যাট কামিন্স। তাঁর পয়েন্ট ৮৯২। দ্বিতীয় স্থানে রয়েছেন বুমরার সতীর্থ রবিচন্দ্রন অশ্বিন। শ্রীলঙ্কার বিরুদ্ধে দু’টেস্টেই ভাল বল করায় তালিকায় উন্নতি হয়েছে তাঁর। অশ্বিনের পয়েন্ট ৮৫০। তিন নম্বরে রয়েছেন দক্ষিণ আফ্রিকার পেসার কাগিসো রাবাডা। তিনি পেয়েছেন ৮৩৫ পয়েন্ট। পাঁচ নম্বরে থাকা পাকিস্তানের পেসার শাহিন শাহ আফ্রিদির পয়েন্ট ৮২২।"
1.1
ARTICLE - 1 => 
{6: 0.097166, 11: 0.8484798}
{5: 0.08498031, 6: 0.09147979, 11: 0.8106145}
0.99999857
 আইসিসি-র সেরার তালিকায় জায়গা পেলেন না কোনও ভারতীয়। বুধবার প্রকাশিত তালিকায়, আইসিসি ক্রিকেটার অফ দি ইয়ার নির্বাচিত হয়েছেন অস্ট্রেলিয়ার স্টিভ স্মিথ। টেস্টেও সেরা তিনিই। স্মিথ হলেন চতুর্থ অস্ট্রেলীয় ক্রিকেটার যিনি এই পুরস্কার জিতলেন।
টেস্টেও সেরা হয়েছে

DO YOU WANT TO CHECK FOR ANOTHER ARTICLE (Y/N) :- n
